In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install emot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/61.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import joblib
from joblib import dump
import dill

# Data Cleaning and preprocessing
import numpy as np
import os
import re
from wordcloud import STOPWORDS
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm


# EDA
import matplotlib.pyplot as plt
import seaborn as sns
#import empiricaldist as emd
import plotly.express as px
from wordcloud import WordCloud

# Modelling
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

#Zero-shot model
#from transformers import pipeline

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, make_scorer, recall_score

In [2]:
# Loading a zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="FacebookAI/roberta-large-mnli",
                      TORCH_USE_CUDA_DSA=1)
#stockbert = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are 

In [3]:
def classify_stock(text):
    # Define candidate labels
    candidate_labels = ["stock market", "other"]

    # Classify the text
    result = classifier(text, candidate_labels=candidate_labels)

    # Return 1 if "stock market" is the predicted label, else 0
    return 1 if result["labels"][0] == "stock market" else 0

def classify_stock_(text):
  # Example usage
  result = stockbert(text)
  result=result[0]
  return 0 if result["label"][0] =="neutral" else 1

In [4]:
%cd drive/MyDrive/Stock_Data

/content/drive/MyDrive/Stock_Data


In [5]:
# Reading all files into single DF
path='./'
all_files=os.listdir(path)

In [6]:
stock_news=pd.read_csv(path+all_files[0])
stock_news.drop('Unnamed: 0', axis=1, inplace=True)

for file in all_files[1:]:
    new_df=pd.read_csv(path+file)
    new_df.drop('Unnamed: 0', axis=1, inplace=True)
    stock_news=pd.concat([stock_news, new_df], axis=0)

In [7]:
stock_news.shape

(3051, 2)

In [8]:

# na check
stock_news.isna().sum()

,0
date,0
content,295


In [9]:
stock_news.dropna(inplace=True)

In [10]:
# Applying model to contents to get the label - it took 17m to predict on 2000+ content
stock_news["label"] = stock_news["content"].apply(classify_stock)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [12]:
stock_news.label.sum()

526

In [25]:
stock_news[stock_news.label==0]['content'].values[30]

"The EU is under pressure from car makers to weaken its CO2 rules, which outlaw sales of new internal ... [+] cumbustion engine vehicles by 2035\nEurope’s automakers face existential threats on many fronts in 2025, but it’s not all bad news and beleaguered investors are offered some hope.\nThe headlines are not reassuring and the key crisis words are CO2, China, tariffs, restructuring, and Germany. Despite overwhelming negativity, investment bank Morgan Stanley and investment researcher Evercore ISI find positives for investors.\nThe most pressing problem concerns European Union rules on carbon dioxide emissions, with bottom-line-destroying fines for the bigger failures. These CO2 rules, to force reluctant Europeans to buy electric vehicles and gradually ban sales of new internal combustion engine vehicles until all new ones disappear by 2035, will have other perverse effects. Manufacturers and politicians are mounting a dogged campaign to have the rules diluted.\nEU Commission Preside

In [26]:
stock_news.to_csv('all_stock_data_labelled_v2.csv', index= False)

In [2]:
# testing regex to remove links
s = '''That's fair. I think I misinterpreted you originally, but now that makes a lot of sense. Thank you for the explanation!|||Maybe in quality of thought and opinions in some cases, but definitely not when it comes to being an independent adult who functions smoothly and has a stable career and remembers to do things like...|||Out-patient is where you go to a therapuetic program during the day while still living at home, while in-patient is either a) mental hospitals or b) rehab or c) residential treatment centers, which...|||Professionally diagnosed with Major Depressive Disorder, Generalized Anxiety Disorder, PTSD and Body Dysmorphic Disorder.  I'm a trainwreck, I know.|||Sorry to revive a slightly old post, but typing this series has been nagging the hell out of me. Initially I assumed Saikawa was an INTP and Magata an INTJ, but after completing the series I actually...|||I tested as an INFJ for... some reason. I actually bounced between nearly every feeler type for years before I studied the functions and realized that me being overemotional doesn't mean I'm a...|||I think a lot of people mistype in general, but I do agree that INTJ does seem like a desirable type for your average socially awkward nerd to want to be.  At the same time, however, I feel like...|||No, I don't normally get bothered by feelers. But I also seem to be lot more in touch and skillful with my inferior Fe than most IxTPs (just going by the internet because I know like, one IxTP in...|||Gotta learn how to camouflage as a Ne dom in social situations. Really the only time you can use Ti or Si socially is one on one with people you are very close with--and as an INTP who is honestly...|||I'm a respectful artist, apparently.  432898  dat confidence score tho.   Don't know how accurate this thing is over all. I feel like my trust would be lower and my confidence slightly higher,...|||I also would third that Amy seems like more of an ISFP to me, but it's been a while since I've seen the show so I could be totally wrong (and I've only seen season 1 and like the first few episodes...|||I hate smelling bad. I never go a day without a shower and always carry deodorant and perfume in my purse just in case. You guys are gross. I always thought this stereotype was false but guess not!|||Don't have too many thoughts other than terrorism is always awful, but Isis and other Islam practicing terrorist groups actually represent a tiny percent of Muslims as a whole (as Islam is honestly a...|||INTP female.  54.167 out of 100 masculine points, 55.833 out of 100 feminine points, and 67.5 out of 100 androgynous (neutral) points.|||The thing about astrology most people don't realize is that it's actually incredibly complicated and unique to every person. You aren't just a Capricorn; that's just your sun sign. For everyone's...|||INFP   https://www.youtube.com/watch?v=xI6vUbKJ24g|||It's not impossible at all! In fact, it's completely logical when you compare auxiliary Ti to a 5's fixation on gathering and developing knowledge. They're quite similar concepts.  Also, IMHO,...|||What do you guys think about Minnesota? I grew up there and I know that at least from my impression, Canada and Minnesota always seemed strikingly similar in terms of stereotypes and culture. However...|||Ne One Like this? Don't know why I added the underscore, seems a little silly in hindsight. Thank you for your help!|||I do as well! But if I'm only relying on my (not too great) Fe for too long I can eventually come off really uncomfortable and stilted and kind of like a pushover. Once I can get comfortable enough...|||But see, this is actually a huge flaw in our education system and in our socially enforced measurement of intelligence, because intelligence is inherently unmeasurable and impossible to score or...|||@Ne_One I agree with you, I think 'feel' isn't uncommonly used not in the literal sense, but as you mentioned, as a synonym for describing the experience of something or the sensation of something. I...|||INFJ. Vampires feel so Ni to me.|||From your description I'd definitely say you seem to use Fi over Fe. You mentioned that your external behavior seems more Fe but I really wouldn't stress on that too much. Fi is all about personal...|||I'm not sure how you could come to that conclusion if you were doing cognitive function studies as INTPs and INTJs have very different function stacks? You're looking at Ni-Te vs. Ti-Ne, which are...|||Alright! Literally just joined this forum butx85 why not, right?  Having Ti isnx92t really a feeling, Ix92d say. I thought it was a very very normal way of experiencing things until
I eventually came to...@jjdj3$$!!w'w www.helpme.in'''
patt = re.compile(r'(https?:/?/?|ftp:/?/?|www|[yY]outube)+?[\w\-\?\=\%.]+?.[\w\!\£\$\%\^\&\*\(\)\_\+\-\=\{\\}\~\[\]\#\:\@\;\'\<\>\?\,\.\/\\]+')
at =  patt.finditer(s)

for a in at:
    print(a.group(0))

https://www.youtube.com/watch?v=xI6vUbKJ24g
www.helpme.in


In [3]:
all_stock_labeled=pd.read_csv('all_stock_data_labelled_v2.csv')

In [4]:
all_stock_labeled

,date,content,label
0,2025-01-05,Business Insider Edition \nGetting a job was t...,0
1,2025-01-05,Login\nOur new print issue is a definitive tak...,0
2,2025-01-05,FABRICE COFFRINI/Getty Images\nA wave of new t...,0
3,2025-01-05,Business Insider Edition \nEven before the Sur...,0
4,2025-01-05,Downside stock moves were below normal for 202...,0
...,...,...,...
2751,2025-01-30,The world’s most valuable company reported its...,1
2752,2025-01-30,Y Combinator-backed startup Martin AI secures ...,0
2753,2025-01-30,\nGameSpot may receive revenue from affiliate ...,0
2754,2025-01-30,If someone told you that stock market investor...,0


#### OTHER TEXT DATA PREPROCESSING, REPLACING EMOJIS AND EMOTICONS WITH THEIR MEANINGS.

In [5]:
# The code below basically adds escape characters  \\ to the emoticon characters since regex is used to
# remove them.
# for example, we have :-) but ) is a group character in regex so, we had to escape it
new_string = ''
new_emoticons = []
meanings = []
to_escape = [']', '[', '+', '*', '(', ')', '?', '{', '}', '|', '.']
for a, mean in EMOTICONS_EMO.items():
    meanings.append(mean)
    #print(a)
    for each in a:
        if each in to_escape:
            new_string +=  '\\'+each
        else:
            new_string+=each
    new_emoticons.append(new_string)
    new_string = ''

# the problem here was that (\\\\) doesn't escape the second parenthesis, so i had to change it to \\)
for ind, new in enumerate(new_emoticons):
    new_emoticons[ind] = new.replace('(\\\\)', '(\\)')

#### DATA CLEANING FUNCTIONS

In [6]:
def emoji_to_text(text):
    '''This function replaces emojis with their meanings'''
    for emoji, meaning in UNICODE_EMOJI.items():
        text = text.replace(emoji, meaning)
    return text

def convert_emoticons(text):
    ''' This function replaces emoticons which are basically symbols to represent expressions with their meaning
    some are in the contents so it makes sense to remove them.'''
    for emoticon, meaning in zip(new_emoticons, meanings):
      #basically emoticon becomes a pattern here that's why it's important to escape those characters
        text = re.sub(emoticon, meaning, text)
    return text

def remlinks_symbs(df):
    '''This function creates a new column with the links, mentions, symbols, numbers of more than 1character
    removed and emojis, emoticons and replaced with their meaning in the data set'''
    df['content'] = df.content

    # replacing link with space
    # we noticed some links don't have :// it's just https:youtu... and some start from www.
    # example www.youtube.com/watch?v=2o5FHtUaKNQ so we had to tweak the regex multiple times to meet the
    # needs.
    df.content.replace(r'(https?:/?/?|ftp:/?/?|www|[yY]outube)+?[\w\-\?\=\%.]+?.[\w\!\£\$\%\^\&\*\(\)\_\+\-\=\{\\}\~\[\]\#\:\@\;\'\<\>\?\,\.\/\\]+', ' ', regex= True, inplace= True)

    # replacing mentions with space
    df.content.replace(r'@[\w\!\£\$\%\^\&\*\(\)\_\+\-\=\{\\}\~\[\]\#\:\@\;\'\<\>\?\,\.\/\\]+', ' ',
                    regex= True, inplace= True)

    #reducing all to lowercase
    df['content'] = df['content'].str.lower()

    # Replacing emoticons with their meaning
    df.content = df.content.apply(convert_emoticons)

    #replacing emojis with their meanings
    df.content = df.content.apply(emoji_to_text)

    # replacing one or more symbols except ', $, % and _ with space
    df.content.replace(r'[^\w$%_\']+', ' ', regex = True, inplace = True)

    #removes numbers longer than two since 4w3 has a meaning removing numbers will leave just w
    #df.content.replace(r'\d\d+', '', regex = True, inplace = True)

    #replacing underscores with nothing
    #df.content.replace(r'_', '', regex = True, inplace = True)
    return df

# removing apostrophe from beginning and end of contents and leaving them in some words so as to remove
# stopwords.
def rem_pos(content):
    if content[0] == '\'' and content[-1]=='\'':
        content = content[1:-1]
    elif content[0] == '\'':
        content = content[1:]
    elif content[-1]=='\'':
        content = content[:-1]
    else:
        pass
    return content


In [6]:
%%time
remlinks_symbs(all_stock_labeled)

CPU times: total: 8.92 s
Wall time: 9.48 s


,date,content,label
0,2025-01-05,business insider edition getting a job was tou...,0
1,2025-01-05,login our new print issue is a definitive take...,0
2,2025-01-05,fabrice coffrini getty images a wave of new ta...,0
3,2025-01-05,business insider edition even before the surge...,0
4,2025-01-05,downside stock moves were below normal for 202...,0
...,...,...,...
2751,2025-01-30,the world s most valuable company reported its...,1
2752,2025-01-30,y combinator backed startup martin ai secures ...,0
2753,2025-01-30,gamespot may receive revenue from affiliate a...,0
2754,2025-01-30,if someone told you that stock market investor...,0


In [7]:
#testing
re.sub(r'[^\w,$%_]+', ' ', 'I $am, ><??>>#%%a>> junior')

'I $am, %%a junior'

In [7]:
stopwords_better = list(STOPWORDS)
for remove in ['\'', '^', '^^', 'doe', 'ha', 'wa', '\'i', '\'the', '\'s', '\'cause', '\'em']:
    stopwords_better.append(remove)

In [8]:
#import nltk
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('omw-1.4')

In [9]:
for a in stopwords.words('english'):
    stopwords_better.append(a)

In [10]:
# Using stopwords from two different modules, we created a better stopwords list
stopwords_better = list(set(stopwords_better))

In [12]:
def lemmatizee(the_list):
    # Lemmatizer function using the returning default part of speech(noun) for each word passed through it
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(a) for a in the_list.split()]

def lemmatized(sentence):
     # Lemmatizer function using parts of speech tagging.
    lemmatizer = WordNetLemmatizer()
    wnl = WordNetLemmatizer()
    lemmatized = []
    for word, tag in pos_tag(sentence.split()):
        if tag.startswith("NN"):
            lemmatized.append(wnl.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):
            lemmatized.append(wnl.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):
            lemmatized.append(wnl.lemmatize(word, pos='a'))
        else:
            lemmatized.append(word)
    return lemmatized

def BOW(train_data, test_data, ngrams= 1, max_feat= 300, split_data = True):
    '''This does all cleaning and returns the bag of documents vector for both train and test data
    note that you have to manually supply train and test data'''

    # for train_data
    # here we apply the clean function above
    train_data = remlinks_symbs(train_data)

    #for test_data
    test_data = remlinks_symbs(test_data)

    if ngrams == 1:
        #train_data.content.replace(r'[^\w^\']+', ' ', regex = True, inplace = True)
        #train_data.content = train_data.content.apply(rem_pos)

        #test_data.content.replace(r'[^\w^\']+', ' ', regex = True, inplace = True)
        #test_data.content = test_data.content.apply(rem_pos)

        # tokenizing words and making the vectors
        vectorizer=TfidfVectorizer(max_features= max_feat, stop_words= stopwords_better, tokenizer=
                                   lemmatized, token_pattern= '(?u)\\b\\w+\'\w+\\b')
        # the reason it tokenizes "'cause" is because ' is recognized as a boundary

        char_array_tr = vectorizer.fit_transform(train_data.content).toarray()
        
        with open("models/tfidf_vectorizer_300.dill", "wb") as f:
            dill.dump(vectorizer, f)
        #dump(vectorizer, f"models/tfidf_vectorizer_{max_feat}.joblib")
        char_array_te = vectorizer.transform(test_data.content).toarray()

        # Turning into dataframes
        frequency_matrix_tr = pd.DataFrame(char_array_tr, columns= vectorizer.get_feature_names_out())
        frequency_matrix_te = pd.DataFrame(char_array_te, columns= vectorizer.get_feature_names_out())
    else:
        pass
    if split_data== True:
        return (frequency_matrix_tr, frequency_matrix_te)
    else:
        return frequency_matrix_tr




In [13]:
X_train, X_test, y_train, y_test = train_test_split(all_stock_labeled.drop(['label'], axis= 1), 
                                                    all_stock_labeled.label,
                                                    test_size=0.3, random_state=42,
                                                    stratify= all_stock_labeled.label)


In [14]:
%%time
# Here we just call the bow function on xtrain and xtest and it does the cleaning and vectorizing.
train_test = BOW(X_train, X_test, split_data= True)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: total: 2min 38s
Wall time: 2min 45s


In [15]:
train_vector = train_test[0]

In [24]:
test_vector = train_test[1]

In [25]:
test_vector.head()

,0,000,1,10,2,2023,2024,2025,3,4,...,way,week,well,without,work,world,worth,write,year,yield
0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
1,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.104999,0.044753,0.000000,0.000000,...,0.0,0.122716,0.055272,0.0,0.000000,0.0,0.0,0.000000,0.267277,0.000000
2,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.043404,0.000000,0.000000,...,0.0,0.000000,0.053605,0.0,0.107924,0.0,0.0,0.000000,0.037031,0.087398
3,0.121712,0.0,0.060072,0.0,0.030287,0.070883,0.027942,0.119098,0.031994,0.097566,...,0.0,0.000000,0.029418,0.0,0.000000,0.0,0.0,0.000000,0.223545,0.095927
4,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.026247,0.000000,0.000000,...,0.0,0.000000,0.032415,0.0,0.032631,0.0,0.0,0.043064,0.089572,0.000000


## MODELLING

In [19]:
import cloudpickle

with open("models/tfidf_vectorizer_300.dill", "rb") as f:
    tfidf_loaded = dill.load(f)


with open("models/tfidf_vectorizer_300.pkl", "wb") as f:
    cloudpickle.dump(tfidf_loaded, f)

In [29]:
def class_report(y_true, y_pred):
    report = classification_report(y_true, y_pred, target_names=[0,1], output_dict= True)
    #report_df = pd.DataFrame(report).transpose()
    #n = ['s', 'r']#list(labenc.inverse_transform([int(c) for c in list(report_df.index[:16])]))
    #n.append('accuracy')
    #n.append('macro avg')
    #n.append('weighted avg')
    #report_df.index = n
    return report

# XGBOOST

In [69]:
xg = XGBClassifier()

In [70]:
%%time
xg.fit(train_vector, y_train)

CPU times: total: 5.02 s
Wall time: 1.51 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [71]:
xg_pred = xg.predict(test_vector)

In [72]:
class_report(y_test, xg_pred)

{0: {'precision': 0.8859154929577465,
  'recall': 0.9402092675635276,
  'f1-score': 0.9122552574329225,
  'support': 669},
 1: {'precision': 0.6581196581196581,
  'recall': 0.4873417721518987,
  'f1-score': 0.5599999999999999,
  'support': 158},
 'accuracy': 0.8536880290205562,
 'macro avg': {'precision': 0.7720175755387023,
  'recall': 0.7137755198577131,
  'f1-score': 0.7361276287164612,
  'support': 827},
 'weighted avg': {'precision': 0.8423946442220538,
  'recall': 0.8536880290205562,
  'f1-score': 0.8449561876936217,
  'support': 827}}

In [85]:
# Tuning xgboost
xg_param = {'max_depth':[3, 10, 15], 'learning_rate':[0.05, 0.1, 0.3], 'gamma':[0.2, 0.4], 'scale_pos_weight':[15,20,25]}

In [86]:
xg_grid = RandomizedSearchCV(XGBClassifier(n_jobs= -1), xg_param, cv= 2)

In [87]:
xg_grid.fit(train_vector, y_train)

RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_cat_to_onehot=None,
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=-1,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   param_distributions={'gamma': [0.2, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.3],
                                        'max_depth': [3, 10, 15],
                                        'scale_pos_weight': [15, 20, 25]})

In [88]:
xg_grid.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [89]:
grid_pred = xg_grid.predict(test_vector)

In [90]:
class_report(y_test, grid_pred)

{0: {'precision': 0.9025110782865583,
  'recall': 0.9133034379671151,
  'f1-score': 0.9078751857355126,
  'support': 669},
 1: {'precision': 0.6133333333333333,
  'recall': 0.5822784810126582,
  'f1-score': 0.5974025974025974,
  'support': 158},
 'accuracy': 0.8500604594921403,
 'macro avg': {'precision': 0.7579222058099457,
  'recall': 0.7477909594898866,
  'f1-score': 0.752638891569055,
  'support': 827},
 'weighted avg': {'precision': 0.8472630931564379,
  'recall': 0.8500604594921403,
  'f1-score': 0.8485587782910137,
  'support': 827}}

## RandomForestClassifier.

In [23]:
rand_f = RandomForestClassifier()

In [24]:
rand_f.fit(train_vector, y_train)

RandomForestClassifier()

In [25]:
rf_pred = rand_f.predict(test_vector)

In [26]:
class_report(y_test, rf_pred)

{0: {'precision': 0.8695652173913043,
  'recall': 0.9865470852017937,
  'f1-score': 0.9243697478991596,
  'support': 669},
 1: {'precision': 0.8676470588235294,
  'recall': 0.37341772151898733,
  'f1-score': 0.5221238938053098,
  'support': 158},
 'accuracy': 0.8694074969770254,
 'macro avg': {'precision': 0.8686061381074168,
  'recall': 0.6799824033603905,
  'f1-score': 0.7232468208522347,
  'support': 827},
 'weighted avg': {'precision': 0.8691987493698915,
  'recall': 0.8694074969770254,
  'f1-score': 0.8475198749283878,
  'support': 827}}

In [35]:
# tuning rf
# Initialize the RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Set up the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Depth of the trees
    'min_samples_split': [2, 5, 10],  # Min samples required to split an internal node
    'min_samples_leaf': [2, 4, 8],  # Min samples required at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at each split
    'bootstrap': [True, False]  # Whether to use bootstrap sampling
}

# Set up GridSearchCV with the Random Forest Classifier
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='recall', n_jobs=-1, verbose=2)

# Fit the model using GridSearchCV
grid_search.fit(train_vector, y_train)

# Print out the best parameters and score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Precision Score: {grid_search.best_score_}")

# Evaluate on the test set
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(test_vector)
print(f"Test Precision: {precision_score(y_test, y_pred)}")

Fitting 3 folds for each of 648 candidates, totalling 1944 fits


C:\Users\User\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Parameters: {'bootstrap': False, 'max_depth': 30, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best Precision Score: 0.37767115376071797
Test Precision: 0.8181818181818182


In [62]:
cm=confusion_matrix(y_test, lgb_pred)
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
cm_df

,Predicted 0,Predicted 1
Actual 0,560,109
Actual 1,48,110


In [30]:
recall_score(y_test, y_pred)

0.7278481012658228

In [34]:
15/16

0.9375

In [26]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_vector, y_train)
gnb_pred = gnb.predict(test_vector)

In [27]:
%%time
# Define the Gaussian Naïve Bayes model
gnb = GaussianNB()

# Define hyperparameter grid
param_grid = {
    "var_smoothing": np.logspace(-100, -2, 50)  # Search across a log scale
}

# Define scoring metric as precision
scorer = make_scorer(f1_score)

# Perform GridSearchCV
grid_search = GridSearchCV(gnb, param_grid, scoring=scorer, cv=5, n_jobs=-1)
grid_search.fit(train_vector, y_train)

# Best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Recall Score: {grid_search.best_score_:.4f}")

# Evaluate on test set
best_gnb = grid_search.best_estimator_
y_pred = best_gnb.predict(test_vector)
recall = recall_score(y_test, y_pred)

print(f"Test Set Recall: {recall:.4f}")

Best Parameters: {'var_smoothing': 0.01}
Best Recall Score: 0.4990
Test Set Recall: 0.7278
CPU times: total: 1.12 s
Wall time: 11.5 s


In [ ]:
dump(best_gnb, "models/naive_bayes.joblib")

In [31]:
class_report(y_test, y_pred)

{0: {'precision': 0.9148514851485149,
  'recall': 0.6905829596412556,
  'f1-score': 0.787052810902896,
  'support': 669},
 1: {'precision': 0.35714285714285715,
  'recall': 0.7278481012658228,
  'f1-score': 0.4791666666666667,
  'support': 158},
 'accuracy': 0.6977025392986699,
 'macro avg': {'precision': 0.635997171145686,
  'recall': 0.7092155304535392,
  'f1-score': 0.6331097387847814,
  'support': 827},
 'weighted avg': {'precision': 0.8083001390482804,
  'recall': 0.6977025392986699,
  'f1-score': 0.7282305487634473,
  'support': 827}}

## Light Gradient boosting

In [58]:
params = {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'min_child_samples': 25, 'n_estimators': 500, 'num_leaves': 128, 'scale_pos_weight': 30, 'subsample': 0.7}
lgb = LGBMClassifier(**params)

In [59]:
lgb.fit(train_vector, y_train)

LGBMClassifier(colsample_bytree=0.7, learning_rate=0.01, max_depth=10,
               min_child_samples=25, n_estimators=500, num_leaves=128,
               scale_pos_weight=30, subsample=0.7)

In [60]:
lgb_pred = lgb.predict(test_vector)

In [63]:
dump(lgb, 'models/lgb.joblib')

['models/lgb.joblib']

In [61]:
class_report(y_test, lgb_pred)

{0: {'precision': 0.9210526315789473,
  'recall': 0.8370702541106129,
  'f1-score': 0.8770555990602976,
  'support': 669},
 1: {'precision': 0.502283105022831,
  'recall': 0.6962025316455697,
  'f1-score': 0.583554376657825,
  'support': 158},
 'accuracy': 0.8101571946795647,
 'macro avg': {'precision': 0.7116678683008892,
  'recall': 0.7666363928780913,
  'f1-score': 0.7303049878590613,
  'support': 827},
 'weighted avg': {'precision': 0.8410458780168356,
  'recall': 0.8101571946795647,
  'f1-score': 0.8209816049374553,
  'support': 827}}

In [33]:
%%time
# Define the LightGBM model
lgbm = LGBMClassifier(objective="binary", random_state=42)

# Define parameter grid
param_grid = {
    "num_leaves": [64, 128],  # Controls complexity
    "max_depth": [-1, 10, 20],    # Limits tree depth (-1 means no limit)
    "learning_rate": [0.1, 0.2],  # Step size for gradient descent
    "n_estimators": [100, 300],  # Number of boosting rounds
    "min_child_samples": [10, 20],  # Minimum data in leaf nodes
    "subsample": [0.7, 0.8],  # Fraction of data per tree (regularization)
    "colsample_bytree": [0.7, 0.8],  # Fraction of features per tree (regularization)
    "scale_pos_weight":[20]
}

# Define scoring metric (F1-score)
scorer = make_scorer(recall_score)  # Adjust 'macro' for multi-class cases

# Perform GridSearchCV
grid_search = GridSearchCV(lgbm, param_grid, scoring=scorer, cv=5, n_jobs=-1, verbose=10)
grid_search.fit(train_vector, y_train)

# Best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best F1 Score: {grid_search.best_score_:.4f}")

# Evaluate on test set
best_lgbm = grid_search.best_estimator_
y_pred = best_lgbm.predict(test_vector)
f1 = f1_score(y_test, y_pred)

print(f"Test Set F1 Score: {f1:.4f}")

Fitting 5 folds for each of 192 candidates, totalling 960 fits
Best Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_samples': 20, 'n_estimators': 100, 'num_leaves': 128, 'scale_pos_weight': 20, 'subsample': 0.7}
Best F1 Score: 0.6142
Test Set F1 Score: 0.6025
CPU times: total: 22.3 s
Wall time: 25min 16s


In [34]:
class_report(y_test, y_pred)

{0: {'precision': 0.9079939668174962,
  'recall': 0.8998505231689088,
  'f1-score': 0.9039039039039038,
  'support': 669},
 1: {'precision': 0.5914634146341463,
  'recall': 0.6139240506329114,
  'f1-score': 0.6024844720496895,
  'support': 158},
 'accuracy': 0.845223700120919,
 'macro avg': {'precision': 0.7497286907258213,
  'recall': 0.7568872869009101,
  'f1-score': 0.7531941879767967,
  'support': 827},
 'weighted avg': {'precision': 0.847520173292745,
  'recall': 0.845223700120919,
  'f1-score': 0.8463171200671857,
  'support': 827}}